# Dependencies

In [1]:
import pandas as pd
import numpy as np
from matplotlib import pyplot as plt
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

# Data Review and Cleaning

In [2]:
file = "Resources/cleaned_drug_data.csv"
df = pd.read_csv(file)
df.head()

,ID,Age,Gender,Education,Country,Ethnicity,Nscore,Escore,Oscore,AScore,...,Legalh,LSD,Meth,Mushrooms,Nicotine,VSA,illegal_score_sum,legal_score_sum,illegal_use,legal_use
0,2,25-34,M,Doctorate degree,UK,White,-0.67825,1.93886,1.43533,0.76096,...,0,0,1,0,1,0,3,4,1,1
1,3,35-44,M,Professional certificate/ diploma,UK,White,-0.46725,0.80523,-0.84732,-1.62090,...,0,0,0,0,0,0,0,3,0,1
2,4,18-24,F,Masters degree,UK,White,-0.14882,-0.80615,-0.01928,0.59042,...,0,0,0,0,0,0,1,3,1,1
3,5,35-44,F,Doctorate degree,UK,White,0.73545,-1.63340,-0.45174,-0.30172,...,0,0,0,0,0,0,0,3,0,1
4,6,65+,F,Left school at 18 years,Canada,White,-0.67825,-0.30033,-1.55521,2.03972,...,0,0,0,0,1,0,0,3,0,1


In [7]:
df.to_html("data_table.html")

In [ ]:
df.columns

In [ ]:
df = df.drop(["ID", 'Alcohol',
       'Amphet', 'Amyl', 'Benzos', 'Caff', 'Cannabis', 'Choc', 'Coke', 'Crack',
       'Ecstasy', 'Heroin', 'Ketamine', 'Legalh', 'LSD', 'Meth', 'Mushrooms',
       'Nicotine', 'VSA', 'illegal_score_sum', 'legal_score_sum', 'legal_use' ], axis=1)

In [ ]:
df.Age.value_counts()

Note: There are 6 categories for Age

In [ ]:
df.Education.value_counts()

Note: There are 9 categories for Education

In [ ]:
df.Country.value_counts()

Note: There are 7 categories for Country

In [ ]:
df.Ethnicity.value_counts()

Note: There are 7 categories for Ethnicity

In [ ]:
df.isnull().sum()

## Encoding Categorical Columns

In [ ]:
# create list of dtypes to help determine unique values
df_cat=df.dtypes[df.dtypes == "object"].index.tolist()

In [ ]:
# display number of unique values
df[df_cat].nunique()

In [ ]:
#import onehotencoder
from sklearn.preprocessing import OneHotEncoder

# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df = pd.DataFrame(enc.fit_transform(df[df_cat]))

# Add the encoded variable names to the dataframe
encode_df.columns = enc.get_feature_names_out(df_cat)
encode_df

## Merge to Create One Dataset

In [ ]:
# merge original df and encoded and then drop the categorical columns to leave only numerical columns
df_merge = df.merge(encode_df, left_index=True, right_index=True)
df_merge = df_merge.drop(df_cat, axis='columns')
df_merge.head()

## Split Data into Target and Features

In [ ]:
# create dataframe for variable X
df_pre = df_merge.drop("illegal_use", axis = 1)

In [ ]:
# split data
X = df_pre
y = df["illegal_use"]

## Train Test Split and Scale Data

In [ ]:
# Split the data into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state =1)

In [ ]:
# scale data
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

## Random Forest Classifier

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
clf = RandomForestClassifier(random_state=1, n_estimators=100, max_depth=5).fit(X_train_scaled, y_train)
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

In [ ]:
feature_importances = clf.feature_importances_

In [ ]:
features = sorted(zip(X.columns, clf.feature_importances_), key = lambda x: x[1])
cols = [f[0] for f in features]
width = [f[1] for f in features]

fig, ax = plt.subplots()

fig.set_size_inches(10,20)
plt.margins(y=0.001)

ax.barh(y=cols, width=width)

plt.show()

## Logistic Regression

In [ ]:
from sklearn.linear_model import LogisticRegression
clf = LogisticRegression()
clf

In [ ]:
# fit the logisticregression model
clf.fit(X_train, y_train)

In [ ]:
# score the training and test models and print the scores
print(f"Training Data Score: {clf.score(X_train, y_train)}")
print(f"Testing Data Score: {clf.score(X_test, y_test)}")

## Try less features

In [ ]:
df1 = df.drop(['Ethnicity'], axis=1)
df1

In [ ]:
# create list of dtypes to help determine unique values
df1_cat=df1.dtypes[df1.dtypes == "object"].index.tolist()

In [ ]:
# Create a OneHotEncoder instance
enc = OneHotEncoder(sparse=False)

# Fit and transform the OneHotEncoder using the categorical variable list
encode_df1 = pd.DataFrame(enc.fit_transform(df1[df1_cat]))

# Add the encoded variable names to the dataframe
encode_df1.columns = enc.get_feature_names_out(df1_cat)
encode_df1

In [ ]:
# merge original df and encoded and then drop the categorical columns to leave only numerical columns
df1_merge = df1.merge(encode_df1, left_index=True, right_index=True)
df1_merge = df1_merge.drop(df1_cat, axis='columns')
df1_merge.head()

In [ ]:
# create dataframe for variable X
df1_pre = df1_merge.drop("illegal_use", axis = 1)

In [ ]:
# split data
X = df1_pre
y = df["illegal_use"]

In [ ]:
# Split the data into X_train, X_test, y_train, y_test
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state =1)

In [ ]:
# scale data
scaler = StandardScaler().fit(X_train)
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)

In [ ]:
clf = RandomForestClassifier(random_state=42).fit(X_train_scaled, y_train)
print(f'Training Score: {clf.score(X_train_scaled, y_train)}')
print(f'Testing Score: {clf.score(X_test_scaled, y_test)}')

In [ ]:
feature_importances = clf.feature_importances_

In [ ]:
features = sorted(zip(X.columns, clf.feature_importances_), key = lambda x: x[1])
cols = [f[0] for f in features]
width = [f[1] for f in features]

fig, ax = plt.subplots()

fig.set_size_inches(10,20)
plt.margins(y=0.001)

ax.barh(y=cols, width=width)

plt.show()

In [ ]:
clf = LogisticRegression()
clf

In [ ]:
clf.fit(X_train, y_train)

In [ ]:
# score the training and test models and print the scores
print(f"Training Data Score: {clf.score(X_train, y_train)}")
print(f"Testing Data Score: {clf.score(X_test, y_test)}")

In [ ]:
from sklearn.neighbors import KNeighborsClassifier
train_scores = []
test_scores = []
for k in range(1, 20, 2):
    knn = KNeighborsClassifier(n_neighbors=k)
    knn.fit(X_train_scaled, y_train)
    train_score = knn.score(X_train_scaled, y_train)
    test_score = knn.score(X_test_scaled, y_test)
    train_scores.append(train_score)
    test_scores.append(test_score)
    print(f"k: {k}, Train/Test Score: {train_score:.3f}/{test_score:.3f}")
    
    
plt.plot(range(1, 20, 2), train_scores, marker='o')
plt.plot(range(1, 20, 2), test_scores, marker="x")
plt.xlabel("k neighbors")
plt.ylabel("Testing accuracy score")
plt.show()

In [ ]:
knn = KNeighborsClassifier(n_neighbors=5)
knn.fit(X_train_scaled, y_train)
print('k=5 Test Acc: %.3f' % knn.score(X_test_scaled, y_test))